# Домашнее задание 3.
## Классификатор на сезонные ряды для построения автоматического пайплайна прогнозирования.

В рамках курса мы разбирали множество подходов к прогнозированию временных рядов, однако одновременно мы работали со счетным количеством рядов,
для каждого из которых была возможность проанализировать его "вручную" и подобрать наилучшую модель. Но что, если мы столкнулись с задачей прогнозирования
сразу сотен рядов? В таком случае мы уже не можем строить модель на каждый ряд вручную и нам придется либо строить одну модель сразу на все ряды и надеяться, что качество будет удовлетворительным, либо придумывать некий automl подход. 

Если позволяют время и ресурсы, мы можем делать автомл "влоб" - перебирать множество моделей с их гиперпараметрами, и выбирать модель с наилучшим качеством на отложенной выборке.

Другим вариантом automl подхода может служить тот, в котором мы разбиваем временные ряды на типы и для каждого типа строим модель прогнозирования, наиболее хорошо такой тип описывающую. Для того, чтобы автоматически определять тип ряда, мы строим классификатор, способный этот тип определять.

Одним из самых очевидных разбиений рядов на типы может быть разбиение на сезонные и не сезонные ряды. Предположение о наличии сезонности позволит использовать алгоритмы, лучше заточенные под поиск сезонных зависимостей, кроме того, это позволит сэкономить вычислительные ресурсы, так как несезонные ряды зачастую можно хорошо прогнозировать более легковесными методами. 

В текущем задании вам как раз и предлагается спроектировать такой бинарный классификатор, который мог бы отделять сезонные ряды от несезонных.

## Задание

Ниже приведен класс для обучения произвольного классификатора на датасете одномерных временных рядов, т.е. на наборе обьектов вида
(timeseries, label). Данный класс определяет метод для получения произвольного признакового описания ряда `get_feature_vector`, использует его для получения датасета, после чего обучает на датасете бинарный классификатор удовлетворяющий sklearn estimator API.

**Вам необходимо:**
1. Определить метод `get_feature_vector`, который позволил бы выделить из временного ряда характерные признаки, указывающие на сезонность.
2. Разбить датасет в соотношении 60/40 (train, test). Проследите за сбалансированностью классов в выборках.
3. Обучить модель на трейне.
4. Сделайте прогноз на тесте и получите метрики `f1`, `auc-roc`.
5. Итоговые баллы за задание будут зависеть от значения метрик.  
   `50 < f1,roc < 60` 4 балла  
   `60 < f1,roc < 70` 6 баллов  
   `70 < f1,roc < 85` 8 баллов  
   `85 < f1,roc < 98` 9 баллов  
   `98 < f1,roc` 10 баллов  

In [1162]:
from pathlib import Path
from typing import Iterable, Tuple

import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from statsmodels.tsa.stattools import acf
from sklearn.linear_model import LogisticRegression
from sktime.transformations.series.detrend import Detrender
from sklearn.ensemble import GradientBoostingClassifier
%matplotlib inline

class SeasonalClassifier:
    def __init__(self, classifier=LogisticRegression, **kwargs):
        self.classifier = classifier(**kwargs)
        self.fitted = False

    def fit(self, ts_dataset: Iterable[Tuple[pd.Series, int]], **kwargs):
        X, y = [], []
        for ts, label in ts_dataset:
            feature_vector = self.get_feature_vector(ts)
            X.append(feature_vector)
            y.append(label)
        X = np.array(X)
        y = np.array(y)
        self.classifier.fit(X, y, **kwargs)
        self.fitted = True

    def predict(self, ts: pd.Series):
        self._check_fitted()
        feature_vector = self.get_feature_vector(ts)
        label = self.classifier.predict(np.array([feature_vector]))
        return label

    def predict_proba(self, ts: pd.Series):
        self._check_fitted()
        feature_vector = self.get_feature_vector(ts)
        proba = self.classifier.predict_proba(np.array([feature_vector]))
        return proba

 
    def get_feature_vector(self, ts: pd.Series):
        feature_vector = []
        
        
        #ts = ts.values
        det = detr.fit_transform(ts) #Убираю тренд тз ряда
        
        fourie = np.abs(np.fft.rfft(det))
        
                
        ts_std = ts.std()
        
        if ts_std > 0:
            ts_acf = acf(det, nlags=len(det))[1:]
            max_idx = np.argmax(acf)
        else:
            ts_acf = np.zeros_like(det)
        max_acf, min_acf = ts_acf.max(),ts_acf.min() 
        max_idx = len(ts)
        
 

        #acf 
        feature_vector.append(firstmin_ac(ts_acf))
        feature_vector.append(ac_first_zero(ts_acf))
        feature_vector.append(max_acf)
        feature_vector.append(min_acf)
        feature_vector.append(max_idx)
        feature_vector.append(first_pos_acf(ts_acf))
        
        #four
        feature_vector.append(fourie.max())
        
        
        #ststist
        
        
        detr_min = det.min()
        feature_vector.append(detr_min)
        feature_vector.append(det.max())
        feature_vector.append(det.mean())
        feature_vector.append(det.std())
        feature_vector.append(np.median(det))
        
        min_count = len(det) - len(det[det == detr_min])
        feature_vector.append(min_count)
        
        #sktime
        feature_vector.append(mean3_stderr(ts))
        return feature_vector

    def _check_fitted(self):
        if not self.fitted:
            raise ValueError('This instance is not fitted yet. Call fit method first.')

In [7]:
# код для считывания датасета
# !ВАЖНО! Не забудьте распаковать датасет перед запуском ячейки

with open('dataset_clf/dataset_clf/labels.csv', 'r') as f:
    labels = (line.replace('\n', '').split(',') for line in f)
    labels = dict(labels)
    labels = {k: int(v) for k, v in labels.items()}

dataset = []

for filename in Path('dataset_clf/dataset_clf/').glob('[!labels]*'):
    ts = pd.read_json(filename, typ='series')
    dataset.append((ts, labels[filename.name]))

In [782]:
detr = Detrender()

In [889]:
def acf_first_zero(acf):
    """Первое значение acf меньше 0"""
    for i in range(1, len(acf)):
        if acf[i] <= 0:
            return i

    return len(acf)

In [1166]:
def mean3_stderr(ts):
    if len(ts) - 5 < 5:
        return 0
    res = _local_simple_mean(ts, 5)
    return np.std(res)

In [1161]:
def first_pos_acf(acf_):
    nsum = 0
    for i in acf_:
        if c > 0:
            nsum+=1
    return nsum

In [891]:
def local_simple_mean(ts: np.ndarray, window):
    res = np.zeros(len(ts) - window)
    for i in range(len(res)):
        nsum = 0
        for n in range(window):
            nsum += ts[i + n]
            print(ts[i + n], nsum)
        res[i] = ts[i + window] - nsum / window
        print(res)
    return res

In [892]:
def firstmin_ac(acf_):
    """Первый минимум acf"""
    for i in range(1, len(acf_) - 1):
        if acf_[i] < acf_[i - 1] and acf_[i] < acf_[i + 1]:
            return i
    return len(acf_)

In [954]:
from itertools import product

def hyperparameters_search(model, param_grid):
    """Подбор гпиерпараметров моделей"""
    statistics_f1 = {}
    statistics_f1['params'] = []
    statistics_f1['f1'] = []

    for param_tuple in product(*param_grid.values()):
        params = dict(zip(param_grid.keys(),param_tuple))


        
        cl = SeasonalClassifier(classifier=model, **params)

        cl.fit(train)
        predict = []
        lab = np.array([data[1] for data in test])
        for i in test:
            predict.append(cl.predict(i[0]))
        pred = np.array(predict).reshape(1,-1)[0]
        
        f1, roc = f1_score(lab,pred), roc_auc_score(lab,pred)
        print(f1, roc)
        statistics_f1['params'].append(params)
        statistics_f1['f1'].append(f1)

        
    best = statistics_f1['params'][np.argmax(statistics_f1['f1'])]
 
    return best

In [1157]:
param_grid = {'n_estimators':[10,20,50]}

In [961]:
b = hyperparameters_search(model = AdaBoostClassifier, )

0.9279279279279279 0.9443575110456554
0.8778409090909091 0.9163751840942562
0.8607594936708861 0.9045471281296024


In [1137]:
split_idx = int(len(dataset)*.6)//2

seas = [(data[0],data[1]) for data in dataset if data[1] ==1]
not_seas = [(data[0],data[1]) for data in dataset if data[1] ==0]

train = seas[:split_idx]+ not_seas[55:split_idx]
test = seas[split_idx:]+ not_seas[split_idx:]

In [1173]:
cl = SeasonalClassifier(classifier=AdaBoostClassifier, n_estimators=10)

cl.fit(train)
predict = []
lab = np.array([data[1] for data in test])
for i in test:
    predict.append(cl.predict(i[0]))


pred = np.array(predict).reshape(1,-1)[0]
f1_score(lab,pred), roc_auc_score(lab,pred)

(0.9279279279279279, 0.9443575110456554)

In [1158]:
from sklearn.metrics import classification_report

In [1174]:
classification_report(lab,pred, output_dict=True)

{'0': {'precision': 0.9605839416058394,
  'recall': 0.9690721649484536,
  'f1-score': 0.9648093841642228,
  'support': 679},
 '1': {'precision': 0.9363636363636364,
  'recall': 0.9196428571428571,
  'f1-score': 0.9279279279279279,
  'support': 336},
 'accuracy': 0.9527093596059113,
 'macro avg': {'precision': 0.9484737889847379,
  'recall': 0.9443575110456554,
  'f1-score': 0.9463686560460753,
  'support': 1015},
 'weighted avg': {'precision': 0.9525661853877309,
  'recall': 0.9527093596059113,
  'f1-score': 0.9526003503756562,
  'support': 1015}}